In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import date,timedelta

# Web scraping preferred over api because of the low rate limits provided by the APIs.
#import weather
#from weather import Weather
#weather = Weather()

# List of New York Locations in the site

In [ ]:
# Locations in New York. Used for finding out all the weather stations' locations. Same approach has to be applied
# for any new location.

locations = ['Albany, New York', 'Alplaus, New York', 'Angus, New York', 'Binghamton, New York', 'Buffalo, New York', 
             'Dansville, New York', 'Ditch Plains, New York', 'Doolins Crossing, New York', 'Dunkirk, New York', 
             'Elmira, New York', 'Farmingdale, New York', 'Fulton, New York', 'Glens Falls, New York', 'Islip, New York', 
             'Ithaca, New York', 'Jamestown, New York', 'Massena, New York', 'Montgomery, New York', 'Monticello, New York', 
             'New York, New York', 'Newburgh, New York', 'Niagara Falls, New York', 'Ogdensburg, New York', 
             'Plattsburgh, New York', 'Poughkeepsie, New York', 'Rochester, New York', 'Saranac Lake, New York', 
             'Shirley, New York', 'Syracuse, New York', 'Utica, New York', 'Watertown, New York', 'Wellsville, New York', 
             'Westhampton, New York', 'White Plains, New York']

places = ['KLGA','KJFK','KJRB','KNYC']

# Generate the Dates

Note: Temporary fix by directly uncommenting the d1 and d2. For more general use, pass the dates to function.

In [ ]:
def Generate_Dates(d1,d2):
    #d1 = date(2008,8,15)
    #d2 = date(2008,9,15)
    
    # List containing all of the dates
    dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
    
    Dates = []
    for d in dd:
        Dates.append((str(d)).replace('-','/'))
        
    return (Dates)

# Creating a function to get the Weather

Function to get the weather of a single location for a single day. Saved in a automatically generated file.

In [ ]:
def Get_Weather(place,date):
    # Create the url from the place and time
    url = 'https://www.wunderground.com/history/airport/' + place + '/' + date + '/DailyHistory.html?req_city=' + place + '&req_statename=New%20York'
    
    # Scrape the HTML at the url
    r = requests.get(url)

    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    
    # Extract the table contents
    data = []
    table = soup.find('table',attrs = {'class':'obs-table responsive'})
    table_body = table.find('tbody')
    rows = table_body.findAll('tr')
    for row in rows:
        cols = row.findAll('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)
        
    # Transfer data to DataFrame
    if (len(cols)==12):
        df = pd.DataFrame(data,columns=['Time_EDT','Temp','Dew_Point','Humidity','Pressure','Visibility','Wind_Dir',
                                        'Wind_Speed','Gust_Speed','Precip','Events','Conditions'])
    elif (len(cols)==13):
        df = pd.DataFrame(data,columns=['Time_EDT','Temp','Windchill','Dew_Point','Humidity','Pressure','Visibility',
                                        'Wind_Dir','Wind_Speed','Gust_Speed','Precip','Events','Conditions'])
    else:
        df = pd.DataFrame(data)
        print ("Size is different. Please check the table")
        
    # Adding date and place to the Dataframe
    Location = []
    Date = []
    for i in range(df.shape[0]):
        Location.append(place)
        Date.append(date)
    df['Location'] = Location
    df['Date'] = Date
    
    # Transfer to file
    df.to_csv(('Weather_' + place + '_' + (date.replace('/','-')) + '.csv'))
    
    return

# Generate the Weather Data

Note: Function generates weather files for the year. Remove the dates generation part by the above defined Generate_Dates() function to get more flexibility with dates.

In [ ]:
def Generate_Data(year):
    # Needed to generate dates
    from datetime import date,timedelta
    
    # Define the start and end dates
    d1 = date(year,1,1)
    d2 = date(year,12,31)
    
    # List containing all of the dates
    dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
    
    Dates = []
    for d in dd:
        Dates.append((str(d)).replace('-','/'))
        
    # List of all the places
    places = ['KLGA','KJFK','KJRB','KNYC']
    
    for date in Dates:
        for place in places:
            try:
                Get_Weather(place,date)
                time.sleep(1)
            except:
                pass

In [ ]:
# To generate data for multiple year, replace by starting year and ending year+1.
for year in range(1997,2007):
    Generate_Data(year)

In [ ]:
# For a single year of data, replace by the year.
Generate_Data(2017)